<a href="https://colab.research.google.com/github/JoshDTT/GAN_MNIST/blob/main/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
# Hiperparámetros
batch_size = 32
learning_rate = 0.0002
num_epochs = 100
image_size = 28 * 28  # MNIST images are 28x28

In [ ]:
# Transformaciones para los datos MNIST
transform = transforms.Compose([
    transforms.ToTensor(),               #convierte los valores de los píxeles (que van de 0 a 255) a un rango de 0 a 1.
    transforms.Normalize((0.5,), (0.5,)) #Esta transformación normaliza los datos para que tengan media 0 y desviación estándar 1,
                                         #una vez transformados, los datos irán de -1 a 1, en lugar de 0 a 1
                                      ]) #terminamos con un tensor

In [ ]:
# Carga del conjunto de datos MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:01<00:00, 5.27MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 153kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1.65M/1.65M [00:01<00:00, 1.45MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.81MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Definición de la red generadora
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(256, 256), #añade una capa con 256 neuronas de entrada y 256 de salida
            nn.ReLU(True),       #función de activación
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, 1024),
            nn.ReLU(True),
            nn.Linear(1024, image_size),
            nn.Tanh()  #Se usa Tanh para que la salida esté entre [-1, 1] para mayor estabilidad y eficiencia
        )

    def forward(self, z):
        return self.model(z)

In [ ]:
# Definición de la red discriminante
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(image_size, 512),
            nn.LeakyReLU(0.2, inplace=True),   #en lugar de truncar los negativos los multiplica por 0.2
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()  # Sigmoid para que la salida esté entre [0, 1]
        )

    def forward(self, x):
        return self.model(x)

In [ ]:
# Inicialización de las redes
generator = Generator()
discriminator = Discriminator()

# Definición de las funciones de pérdida y optimizadores
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=learning_rate)
optimizer_D = optim.Adam(discriminator.parameters(), lr=learning_rate)



In [ ]:
# Entrenamiento
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # Etiquetas para el discriminante
        real_labels = torch.ones(images.size(0), 1)  # Etiquetas para imágenes reales
        fake_labels = torch.zeros(images.size(0), 1)  # Etiquetas para imágenes falsas

        # Convertir imágenes a vectores
        images = images.view(images.size(0), -1)

        # Entrenamiento del discriminante
        optimizer_D.zero_grad()              # reinicia los gradientes del discriminante para cada iteración
        outputs = discriminator(images)
        d_loss_real = criterion(outputs, real_labels)
        d_loss_real.backward()               #tomando d_loss_real realiza backpropagation para ajustar los pesos según la pérdida


        z = torch.randn(images.size(0), 256)  # Ruido aleatorio para generar imágenes diferentes y no partir siempre del mismo punto
        fake_images = generator(z)
        outputs = discriminator(fake_images.detach()) #las imágenes falsas se pasan al discriminante para obtener sus predicciones.
        d_loss_fake = criterion(outputs, fake_labels) #criterion es la función de pérdida, en este caso BCE
        d_loss_fake.backward()

        optimizer_D.step()
        d_loss = d_loss_real + d_loss_fake

        # Entrenamiento del generador
        optimizer_G.zero_grad()
        outputs = discriminator(fake_images)
        g_loss = criterion(outputs, real_labels)
        g_loss.backward()
        optimizer_G.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], '
                  f'D Loss: {d_loss.item():.4f}, G Loss: {g_loss.item():.4f}')


Epoch [1/100], Step [100/1875], D Loss: 0.7613, G Loss: 0.9914
Epoch [1/100], Step [200/1875], D Loss: 0.5297, G Loss: 0.9679
Epoch [1/100], Step [300/1875], D Loss: 0.5367, G Loss: 2.2571
Epoch [1/100], Step [400/1875], D Loss: 0.0587, G Loss: 6.1359
Epoch [1/100], Step [500/1875], D Loss: 0.0670, G Loss: 5.5127
Epoch [1/100], Step [600/1875], D Loss: 0.1610, G Loss: 6.9262
Epoch [1/100], Step [700/1875], D Loss: 0.3051, G Loss: 6.1256
Epoch [1/100], Step [800/1875], D Loss: 0.0128, G Loss: 6.3781
Epoch [1/100], Step [900/1875], D Loss: 0.0136, G Loss: 6.8400
Epoch [1/100], Step [1000/1875], D Loss: 0.1742, G Loss: 9.3117
Epoch [1/100], Step [1100/1875], D Loss: 0.0117, G Loss: 8.3049
Epoch [1/100], Step [1200/1875], D Loss: 0.1633, G Loss: 7.9205
Epoch [1/100], Step [1300/1875], D Loss: 0.0486, G Loss: 6.9481
Epoch [1/100], Step [1400/1875], D Loss: 0.0706, G Loss: 9.9912
Epoch [1/100], Step [1500/1875], D Loss: 0.2757, G Loss: 7.4868
Epoch [1/100], Step [1600/1875], D Loss: 0.2986, 

In [ ]:
# Generación de imágenes
#se crea un tensor z que representa 64 vectores de ruido aleatorio, cada uno con dimensión 100. Sirve como input del generador.
#y es la semilla para crear imágenes.
z = torch.randn(64, 256)


#el ruido  z se pasa al generador el cual lo transforma en imágenes de salida, el output tendrá la forma (64,784) debido a la última capa del generador.
generated_images = generator(z).view(-1, 1, 28, 28).detach().numpy()

#-1 en la primera entrada permite a PyTorch determinar automáticamente el tamaño del primer eje (en este caso, el batch size).
#(1, 28, 28) es el tamaño final de cada imagen e indica que es una imagen de un solo canal (blanco y negro) de tamaño 28x28.
#detach() evita que las operaciones posteriores afecten al grafo de cómputo de PyTorch, congelando las imágenes generadas en su estado actual.
#numpy() convierte el tensor de PyTorch en un arreglo de NumPy para que se pueda visualizar con matplotlib.



# Visualización de las imágenes generadas.
fig, axes = plt.subplots(8, 8, figsize=(10, 10))
for i, ax in enumerate(axes.flat):
    ax.imshow(generated_images[i][0], cmap='gray')
    ax.axis('off')
plt.show()

NameError: name 'generator' is not defined